In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
#set the executable path and initialize a browser.
executable_path = {'executable_path': 'C:/Users/Mobeen/.wdm/drivers/chromedriver/win32/89.0.4389.23/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
def mars_news(browser):
    
#Visit the mars nasa news site
    url = 'https://data-class-mars.s3.amazonaws.com/Mars/index.html'
    browser.visit(url)
#Optional delay for loading the page
    browser.is_element_present_by_css('div.list_text', wait_time=1)

    html = browser.html
    news_soup = soup(html, 'html.parser')
    
    #Add try/except for error handling
    try:
        slide_elem = news_soup.select_one('div.list_text')
        news_title = slide_elem.find('div', class_='content_title').get_text()
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
    except AttributeError:
        return None, None
    
    return news_title, news_p

### Featured Images 

In [8]:
def featured_image(browser):

# Visit Url 
    url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
    browser.visit(url)

# Find and click the full image button 
    full_image_elem = browser.find_by_tag('button')[1] # we want to click on the second button 
    full_image_elem.click()

#Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')

# Find the relative image url 
    try: 
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
    
    except AttributeError: 
        return None
    
    # Use the base url to create an absolute url 
    img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'
    
    return img_url 



# Scrape Mars Data: Mars Facts

In [9]:
def mars_facts(browser): 
    try:
        #use 'read_html' to scrape the facts table into a dataframe 
        df = pd.read_html('https://data-class-mars-facts.s3.amazonaws.com/Mars_Facts/index.html')[0]
    except BaseException:
        return None 
    #Assigns columns and set index of the dataframe 
    df.columns=['description', 'Mars', 'Earth']
    df.set_index('description', inplace=True)
    
    # convert dataframe into HTML format, add bootstrap 
    return df.to_html()


In [10]:
browser.quit()
#To fully automate jupyter notebook need to convert into .py file 